<a href="https://colab.research.google.com/github/Merlinvensiya/Gen-AI/blob/main/human%20faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
(X_train, y_train), (_, _) = tf.keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [3]:
X_train.shape


(60000, 28, 28)

In [4]:
28 * 28

784

In [5]:
y_train.shape

(60000,)

In [7]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')

In [8]:
X_train.shape

(60000, 28, 28, 1)

In [9]:
X_train[0].min(), X_train[0].max()

(0.0, 255.0)

In [10]:
X_train = (X_train - 127.5) / 127.5

In [11]:
X_train[0].min(), X_train[0].max()

(-1.0, 1.0)

In [12]:
import pandas as pd
import numpy as np

from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint

from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.models import load_model

import matplotlib.pyplot as plt
import glob, cv2
import seaborn as sns

import os, shutil
from tqdm import tqdm
from pathlib import Path

In [13]:
def load_data(src_dir="", image_size=224):
    img_data = glob.glob(src_dir + "/*")
    X = []
    for img_path in tqdm(img_data):
        try:
            img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (image_size, image_size), interpolation=cv2.INTER_LINEAR)

            X.append(img)
        except Exception as e:
            pass
#         break

    X = np.array(X)

    print(X.shape)
    return X

In [14]:
image_size = 256
X_train = load_data(src_dir="/kaggle/input/face-data/FaceData/", image_size=image_size)

0it [00:00, ?it/s]

(0,)


In [15]:
def plot_images(images, figsize=(20, 4)):
    num_images = len(images)

    fig, axes = plt.subplots(1, num_images, figsize=figsize)

    for i, ax in enumerate(axes):
        ax.imshow(images[i])
        ax.axis('off')

    plt.show()

In [17]:
if len(X_train) > 0:
    indices = np.random.randint(0, len(X_train), size=10)
    sample_images = [X_train[ind] for ind in indices]

    plot_images(sample_images)
else:
    print("X_train is empty. Cannot generate random indices.")

X_train is empty. Cannot generate random indices.


In [18]:
# define the standalone discriminator model
def define_discriminator(in_shape=(image_size, image_size, 3)):
    model = Sequential()
    # normal
    model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))


    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

In [19]:
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 4x4 image
    n_nodes = 192 * 4 * 4
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4, 4, 192)))
    # upsample to 8x8
    model.add(Conv2DTranspose(192, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 16x16
    model.add(Conv2DTranspose(192, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 32x32
    model.add(Conv2DTranspose(192, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 64x64
    model.add(Conv2DTranspose(192, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 128x128
    model.add(Conv2DTranspose(192, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 256x256
    model.add(Conv2DTranspose(192, (4, 4), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    # output layer
    model.add(Conv2D(3, (3, 3), activation='tanh', padding='same'))

    return model

In [20]:
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [21]:
def load_real_samples(X_train):
    # load cifar10 dataset
    # (trainX, _), (_, _) = load_data()
    # convert from unsigned ints to floats
    X = X_train.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5

    return X

In [22]:
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))

    return X, y

In [23]:
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)

    return x_input

In [24]:
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))

    return X, y

In [25]:
def save_plot(examples, epoch, n=7):
    # scale from [-1,1] to [0,1]
    examples = (examples + 1) / 2.0
    # plot images
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.imshow(examples[i])
        # save plot to file
        filename = './DCGAN_models/generated_plot_e%03d.png' % (epoch+1)
        plt.savefig(filename)
        plt.close()

In [26]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = './DCGAN_models/generator_model_%03d.h5' % (epoch+1)
    g_model.save(filename)

In [27]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('> Epoch:%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
            (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
        # evaluate the model performance, sometimes
        if (i+1) % 5 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [28]:
latent_dim = 100

# create the discriminator
d_model = define_discriminator()
d_model.summary()

# create the generator
g_model = define_generator(latent_dim)
g_model.summary()

# create the gan
gan_model = define_gan(g_model, d_model)
gan_model.summary()

# load image data
# dataset = load_real_samples()
dataset = load_real_samples(X_train)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 64)      1792      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256, 256, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128, 128, 128)     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 128)       147584    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 64, 64, 128)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 256)       2

In [30]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [31]:
def save_plot(examples, n_cols=10, n_rows=6):
    # Set up the figure with the specified number of columns and rows
    # fig, axs = pyplot.subplots(n_rows, n_cols, figsize=(8, 3))
    fig, axs = pyplot.subplots(n_rows, n_cols, figsize=(15, 8))

    # Flatten the axes array for easy iteration
    axs = axs.flatten()

    # Plot images
    for i in range(n_cols * n_rows):
        # Turn off axis
        axs[i].axis('off')
        # Plot raw pixel data
        axs[i].imshow(examples[i, :, :])

    # Adjust layout for better spacing
    pyplot.tight_layout()
    # Save the plot to a file (optional)
    pyplot.savefig('generated_images_plot.png')
    # Show the plot (optional)
    pyplot.show()

In [33]:
!ls ./DCGAN_models/

ls: cannot access './DCGAN_models/': No such file or directory


In [34]:
!pip install tensorflow
!pip install keras

In [35]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from keras import backend as K

In [36]:
model_path = './DCGAN_models/generator_model_200.h5'